In [0]:
# 1. Leer tablas de la capa Silver
silver_business_path = "s3a://lakehouseyelp/silver/business_clean"
silver_review_path = "s3a://lakehouseyelp/silver/review_enriched"

df_business = spark.read.format("delta").load(silver_business_path)
df_review = spark.read.format("delta").load(silver_review_path)

# 2. Unir tablas (Join por business_id)
# Seleccionamos solo las columnas necesarias para optimizar
df_master = df_review.join(
    df_business.select("business_id", "name", "city", "state", "categories"),
    on="business_id",
    how="inner"
)

In [0]:
from pyspark.sql.functions import col, count, when

# Crear KPI: Sentimiento por Ciudad
gold_city_sentiment = df_master.groupBy("city", "state").agg(
    count("*").alias("total_reviews"),
    count(when(col("ai_sentiment") == "POSITIVE", True)).alias("positives"),
    count(when(col("ai_sentiment") == "NEGATIVE", True)).alias("negatives")
)

# Calcular el % de satisfacción
gold_city_sentiment = gold_city_sentiment.withColumn(
    "satisfaction_rate", 
    (col("positives") / col("total_reviews")) * 100
).orderBy(col("total_reviews").desc())

display(gold_city_sentiment)

city,state,total_reviews,positives,negatives,satisfaction_rate
Philadelphia,PA,849,623,226,73.38044758539458
New Orleans,LA,579,436,143,75.30224525043178
Nashville,TN,344,243,101,70.63953488372093
Tampa,FL,291,200,91,68.72852233676976
Tucson,AZ,266,185,81,69.54887218045113
Reno,NV,264,204,60,77.27272727272727
Indianapolis,IN,239,193,46,80.75313807531381
Saint Louis,MO,227,165,62,72.68722466960352
Santa Barbara,CA,187,144,43,77.00534759358288
Edmonton,AB,79,53,26,67.08860759493672


In [0]:
# Guardar en S3 (Capa Gold)
gold_path = "s3a://lakehouseyelp/gold/city_satisfaction_stats"
gold_city_sentiment.write.format("delta").mode("overwrite").save(gold_path)



In [0]:
# REGISTRAR EN EL METASTORE (Para que aparezca en el menú 'Catalog')
spark.sql("CREATE DATABASE IF NOT EXISTS yelp_analytics")
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS yelp_analytics.city_satisfaction 
    USING DELTA 
    LOCATION '{gold_path}'
""")

DataFrame[]

In [0]:
%sql
select *
from yelp_analytics.city_satisfaction

city,state,total_reviews,positives,negatives,satisfaction_rate
Philadelphia,PA,849,623,226,73.38044758539458
New Orleans,LA,579,436,143,75.30224525043178
Nashville,TN,344,243,101,70.63953488372093
Tampa,FL,291,200,91,68.72852233676976
Tucson,AZ,266,185,81,69.54887218045113
Reno,NV,264,204,60,77.27272727272727
Indianapolis,IN,239,193,46,80.75313807531381
Saint Louis,MO,227,165,62,72.68722466960352
Santa Barbara,CA,187,144,43,77.00534759358288
Edmonton,AB,79,53,26,67.08860759493672
